In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import portfolio_management_tools as pmt
from dotenv import load_dotenv

import yfinance as yf
from pandas_datareader import data as pdr


import warnings
warnings.filterwarnings('ignore')

import statsmodels.api as sm
import scipy.stats as stats

In [2]:
yf.pdr_override()

In [150]:
def market_cap_df(stock_list, start, end):
    price_df = pd.DataFrame()
    cap_df = pd.DataFrame()
    data = yf.Tickers(stock_list)
    for i in stock_list:
        price = data.tickers[i].history(start = start, end = end)[['Close']]
        price_df = pd.concat([price_df, price], axis = 1)
        
        shares_df = pd.DataFrame(data.tickers[i].get_shares_full(start = start, end = end), columns = [i]).reset_index()
        shares_df.columns = ['Date', 'Shares']
        shares_df.set_index('Date', inplace = True)
        shares_df = shares_df[~shares_df.index.duplicated(keep = 'first')]
        
        temp_df = pd.merge(price, shares_df, on = 'Date', how = 'left')
        temp_df.fillna(method = 'ffill', inplace = True)
        temp_df.fillna(method = 'bfill', inplace = True)
        temp_df[i] = temp_df['Close'] * temp_df['Shares']
        
        cap_df = pd.concat([cap_df, temp_df[i]], axis = 1)
    price_df.columns = stock_list
    price_df.interpolate(method='linear', limit_direction='forward', axis=0)
    
    cap_last_day = cap_df.iloc[-1:,:]
        
    rf_df = pd.read_csv('Resources_from_others/10_year_tresury_1980_to_2024.csv')
    rf_df.columns = ['Date', 'Risk free rate (daily)']
    rf_df['Date'] = pd.to_datetime(rf_df['Date']).dt.date
    rf_df.set_index('Date', inplace = True)
    rf_df[rf_df['Risk free rate (daily)'] == '.'] = np.nan
    rf_df['Risk free rate (daily)'] = rf_df['Risk free rate (daily)'].astype('float64') / (252 * 100)
    
    tickers = yf.Tickers(['^GSPC'])
    market_return_df = tickers.tickers['^GSPC'].history(start = start, end = end)[['Close']]
    market_return_df = market_return_df.pct_change().dropna()
    market_return_df.reset_index(inplace = True)
    market_return_df.Date = market_return_df.Date.dt.date
    market_return_df.set_index('Date', inplace = True)

    return price_df, cap_df, cap_last_day, rf_df, market_return_df

def market_weight(cap_last_day):
    w_mkt = cap_last_day / cap_last_day.sum(axis = 1).values[0]
    w_mkt['Market Cap'] = 'Market_cap'
    w_mkt.set_index('Market Cap', inplace = True)
    return w_mkt.T

def cov_matrix(price_df):
    return price_df.pct_change().cov()

def lambda_(market_return_df, risk_free_df):
    merged_df = pd.merge(market_return_df, risk_free_df, on = 'Date', how = 'left')
    merged_df['Excess market return'] = merged_df['Close'] - merged_df['Risk free rate (daily)']
    lambd = merged_df['Excess market return'].mean() / (merged_df['Close'].var())
    return lambd

def implied_return(lambda_, cov_matrix, w_mkt):
    index = w_mkt.index
    return pd.DataFrame(lambda_ * np.dot(cov_matrix, w_mkt) * 252, index = index, columns = ['Annual implied returns'])

def create_row(stock_list):
    final_list = np.zeros(len(stock_list))
    
    valid_total_weights = [-100, 0, 100]
    
    while True:
        affected_stocks = input("Please write the number of stocks that are affected: ")
        try:
            affected_stocks = int(affected_stocks)
            if affected_stocks <= 0 or affected_stocks > len(stock_list):
                raise ValueError("Number of affected stocks must be within the range of available stocks.")
        except ValueError as e:
            print(e)
        else:
            break
    print('***********')    
    
    while True:
        positive_weights = 0
        negative_weights = 0
        lst = []
    
        for i in range(affected_stocks):
            while True:  # Loop for stock symbol input
                stock_symbol = input('Please write the affected stock symbol: ')
                if stock_symbol not in stock_list:
                    print('Invalid stock symbol. Please try again.')
                elif stock_symbol in lst:
                    print('You have entered this stock already. Please try again.')
                else:
                    break  # Valid stock symbol, exit this loop
            lst.append(stock_symbol)
            while True:  # Loop for weight input
                try:
                    weight = input('Please enter weight in percentage: ')
                    weight = int(weight)
                    if weight > 100 or weight < -100 or weight == 0:
                        raise ValueError('Invalid weight. Weight must be between -100 and 100, excluding 0.')
                except ValueError as e:
                    print(e)
                else:
                    break
            if weight < 0:
                negative_weights += weight
                index = stock_list.index(stock_symbol)
                final_list[index] = weight
            elif weight > 0:
                positive_weights += weight
                index = stock_list.index(stock_symbol)
                final_list[index] = weight
            print('----------')
        if positive_weights in valid_total_weights and negative_weights in valid_total_weights:
            break
        else:
            print('The sum of positive and negative weights is not valid. Please start over.')
    
    final_list = final_list / 100
    
    return final_list


def l_matrix(stock_list):
    final_matrix = np.array([])
    while True:
        num_of_views = input('How many view are there?: ')
        try:
            num_of_views = int(num_of_views)
            if num_of_views <= 0:
                raise ValueError('Number you have entered is invalid.')
        except ValueError as e:
            print(e)
        else:
            break
    print('********')
    for i in range(num_of_views):
        print(f'View {i + 1}')
        row = create_row(stock_list)
        if i == 0:
            final_matrix = row
        else:
            final_matrix = np.vstack([final_matrix, row])
    
    final_dataframe = pd.DataFrame(final_matrix, columns = stock_list)
    return final_dataframe

In [63]:
# Making S&P 500 list to grab data
snp_df = pd.read_csv('Resources_from_others/S&P_500_constituents.csv')
snp_df.Symbol[snp_df.Symbol == 'BRK.B'] = 'BRK-B'
snp_df.Symbol[snp_df.Symbol == 'BF.B'] = 'BF-B'
snp_tickers = snp_df.Symbol.tolist()
snp_tickers.remove('DAY')

In [64]:
price_df, cap_df, cap_last_day, rf_df, market_return_df = market_cap_df(snp_tickers, start = '2016-01-01', end = '2024-02-18')

In [65]:
last_day = cap_df.iloc[-1:,:]
last_day

,MMM,AOS,ABT,ABBV,ACN,ADBE,AMD,AES,AFL,A,...,WTW,GWW,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZION,ZTS
2024-02-16 00:00:00-05:00,5.043388e+10,1.178885e+10,1.979282e+11,3.133657e+11,2.316610e+11,2.470903e+11,2.809374e+11,1.122968e+10,4.563042e+10,3.951351e+10,...,2.843481e+10,4.644768e+10,1.187401e+10,3.259577e+10,2.982069e+10,3.762028e+10,1.415283e+10,2.606202e+10,6.020938e+09,8.625757e+10


In [147]:
a,b,c,d,e = market_cap_df(stock_list, start = '2016-01-01', end = '2024-02-18')

In [148]:
f = b.iloc[-1:,:]

In [152]:
z = market_weight(f)
y = cov_matrix(a)
x = lambda_(e, d)
w = implied_return(x, y, z)

In [66]:
w_mkt = market_weight(last_day)
cov_matrix = cov_matrix(price_df)
lambda_ = lambda_(market_return_df, rf_df)

In [96]:
imp_return = implied_return(lambda_, cov_matrix, w_mkt).sort_values(by = 'Annual implied returns', ascending = False)

In [ ]:
stock_list = ['GOOG', 'MSFT', 'AMZN', 'AAPL', 'SBUX']

In [75]:
P = matrix

In [76]:
P

,GOOG,MSFT,AMZN,AAPL,SBUX
0,0.73,0.0,0.27,-0.6,-0.4
1,0.00,0.1,0.00,0.9,0.0
2,1.00,0.0,0.00,0.0,0.0


In [137]:
Q = np.array([[-0.17], [0.17], [0.031]])

In [138]:
tau = 1
sigma = price_df[stock_list].pct_change().cov()
imp_return = imp_return.loc[stock_list]

In [139]:
Omega = tau * np.dot(np.dot(P, sigma), P.T)

In [140]:
Omega

array([[ 1.61180104e-04, -3.81137600e-05,  1.11664629e-04],
       [-3.81137600e-05,  3.18264362e-04,  2.16646195e-04],
       [ 1.11664629e-04,  2.16646195e-04,  3.21303325e-04]])

In [141]:
first_term = np.linalg.inv(np.linalg.inv(tau * sigma) + np.dot(np.dot(P.T, np.linalg.inv(Omega)), P))
second_term = np.dot(np.linalg.inv(tau * sigma), imp_return) + np.dot(np.dot(P.T, np.linalg.inv(Omega)), Q)

In [142]:
E_r = np.dot(first_term, second_term)
E_r = pd.DataFrame(E_r, index = stock_list, columns = ['Black_litterman return'])

In [143]:
E_r

,Black_litterman return
GOOG,0.087018
MSFT,0.123917
AMZN,0.071227
AAPL,0.161722
SBUX,0.165671


References:

SNP_tickers data : https://github.com/datasets/s-and-p-500-companies/blob/main/data/constituents.csv

In [145]:
imp_return

,Annual implied returns
GOOG,0.143036
MSFT,0.147345
AMZN,0.142442
AAPL,0.145720
SBUX,0.116343


In [154]:
w

,Annual implied returns
GOOG,0.194160
MSFT,0.201693
AMZN,0.216416
AAPL,0.203008
SBUX,0.122301


In [ ]:
create_row(stock_list)

In [70]:
matrix = l_matrix(stock_list)

How many view are there?:  3


********
View 1


Please write the number of stocks that are affected:  4


***********


Please write the affected stock symbol:  GOOG
Please enter weight in percentage:  73


...

----------
View 3


Please write the number of stocks that are affected:  1


***********


Please write the affected stock symbol:  GOOG
Please enter weight in percentage:  100


----------


,GOOG,MSFT,AMZN,AAPL,SBUX
0,0.82,0.0,0.18,-0.7,-0.3
1,0.00,0.0,0.00,-0.7,-0.3
2,0.60,0.4,0.00,0.0,-1.0
